In [1]:
import os
import cv2
import imageio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def display_video_from_frames(input_folder, output_path, frame_rate = 20):
    images = []
    for filename in os.listdir(input_folder):
        if filename.endswith('.npy'):
            img = np.load(os.path.join(input_folder, filename))
            img[img > 0] = 255
            img = (img - np.min(img)) / (np.max(img) - np.min(img))
            images.append(img)
    imageio.mimsave(output_path, images, fps=frame_rate)
    return

In [182]:
def process_frames_to_vid(input_folder, output_path, frame_rate=20, persist=0, average=0, lamb=1.0, threshold=None):
    images = []
    num_instances = 0
    # load video and retrieve number of instances
    for filename in os.listdir(input_folder):
        if filename.endswith('.npy'):
            img = np.load(os.path.join(input_folder, filename))
            num_instances = max(num_instances, np.max(img))
            images.append(img)        
    num_instances = int(num_instances)
    print(f'Number of instances in video {input_folder}', num_instances)
        
    # process with persistence
    if persist > 0:
        instance_seen_last = [-1] * num_instances
        for index in range(len(images)):
            image = images[index]
            for i in range(num_instances):
                instance_id = i+1
                if instance_id in image:
                    instance_seen_last[i] = index
                elif instance_seen_last[i] != -1 and index - instance_seen_last[i] <= persist:
                    prev_frame = images[instance_seen_last[i]]
                    mask = (prev_frame == instance_id)
                    curr_vals = images[index][mask].copy()
                    images[index][mask] = np.maximum(curr_vals, instance_id)

    # save as binary mask
    for index in range(len(images)):
        images[index][images[index] > 0] = 255
        
    # run weighted average of frames (no longer binary mask)
    averaged_images = []
    if average > 0:
        weights = np.array([np.exp(-lamb * i) for i in range(average-1, -1, -1)])
        print('Weights:', weights)
        for index in range(len(images)):
            if index < average: 
                frames_to_avg = np.array(images[:index+1])
                weighted_avg = np.average(frames_to_avg, axis=0, weights=weights[average-index-1:])
            else: 
                frames_to_avg = np.array(images[index-average+1:index+1])
                weighted_avg = np.average(frames_to_avg, axis=0, weights=weights)
            
            if threshold is None: 
                averaged_images.append(weighted_avg.astype(np.uint8))
            else: 
                pixel_threshold = threshold * 255
                averaged_images.append((weighted_avg > pixel_threshold).astype(np.uint8) * 255)
        images = averaged_images
    
    imageio.mimsave(output_path, images, fps=frame_rate)
    return images

In [ ]:
arms = process_frames_to_vid('../data/DEVA_masks/arm/', './arms.mp4', persist=20, average=20, lamb=0.8)
objects = process_frames_to_vid('../data/DEVA_masks/utensil.food.kitchen_appliance.pot.pan.knife.cutting_board/', './objects.mp4', persist=10, average=10, lamb=0.6)
scene = process_frames_to_vid('../data/DEVA_masks/door.window', './scene.mp4', persist=10, average=10, lamb=0.5)

In [ ]:
combined_frames = []
for frame_index in range(np.shape(arms)[0]):
    # set arm_frame to be 128 wherever arms is 255
    arm_frame = arms[frame_index]
    objects_frame = objects[frame_index] * 0.5
    scene_frame = scene[frame_index] * 0.8
    combined_frame = np.maximum(np.maximum(arm_frame, objects_frame), scene_frame)
    combined_frames.append(combined_frame.astype(np.uint8))
imageio.mimsave('./combined.mp4', combined_frames, fps=20)